# Steganography

The goal is to hide a secret message in a jpeg. Steps we need to take:

1. Load an image as an array of bytes
2. Convert message into bits
3. For each bit in message, change the lsb of every pixel until the entire message is encoded

*Optional:* 
- Ensure the image is large enough to contain the message. 
- Choose how many bits to change per pixel
  - Distribute bit change equally on all channels

That's it for **encoding** the message, but we also need to **decode** it. To do that, we need to know the size of the message, so we need to make sure the message contains some header. 

With that out of the way, this is the image we will be working on:

<img src="nomorefanmail.jpg" width="500">

First we need to **encode the message in binary**:

In [34]:
def str_to_bin(str : str) -> str:
    bitstring = ''.join(format(ord(char), '08b') for char in msg) # https://www.geeksforgeeks.org/python/python-convert-string-to-binary/
    return bitstring

def bin_to_str(str : str) -> str:
    chars = [chr(int(str[i:i+8], 2)) for i in range(0, len(str), 8)]
    original_msg = ''.join(chars)
    return original_msg

msg = "War and hate"
encoded = str_to_bin(msg)
decoded = bin_to_str(encoded)

print(f"Original message: {msg}")
print(f"Encoded: {encoded}")
print(f"Decoded: {decoded}")

print(len("hell"))

Original message: War and hate
Encoded: 010101110110000101110010001000000110000101101110011001000010000001101000011000010111010001100101
Decoded: War and hate
4


Then, we'll **load the image**:

In [35]:
from PIL import Image
import numpy as np

img = Image.open("nomorefanmail.jpg") 
pixels = np.array(img, dtype=np.uint8) # this gives us a 2D array, pixels[0][0] gives us the data of the first pixel
# example pixel: [224 238 241]


We'll define a function that takes a pixel, and hides one bit in one of the channels.

In [ ]:
def encode_on_pixel(pixel, msg_bit: bool):
    blue_channel = pixel[2]
    binary_str = f'{blue_channel:08b}' # Assuming the channel consists of 8 bits
    #### STRINGS IN PYTHON ARE IMMUTABLE, THIS WONT WORK
    if msg_bit:
        binary_str[-1] = "1"
    else:
        binary_str[-1] = "0"

    


Now we'll call this function on all pixels, until the message is encoded.

In [ ]:
def encode_message(pixels, msg):

    bit_index = 0

    for row in pixels:
        for pixel in row:
            for i in range(bit_index, len(msg)):
                bit = msg[bit_index]
                if bit == "1":
                    bit = True
                elif bit == "0":
                    bit = False
                else:
                    raise ValueError(f'Unexpected bit value in message: \nbit = {bit} \nmsg = {msg}')
                
                
        
        